In [ ]:
import pandas as pd
import csv

import pyspark
import findspark
findspark.init()
import re
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
spark = SparkSession.builder.appName('Homework').getOrCreate()

In [ ]:
amazon_data = pd.read_csv('C:/Users/Sachin Chavan/Downloads/Amazon_Responded_Oct05.csv') #importing the dataset
amazon_data.head() #viewing the dataset

In [ ]:
print(amazon_data.shape) #checking the number of rows and columns

In [ ]:
print(amazon_data.columns) #checking column names

In [ ]:
print(amazon_data.info()) #gathering information regarding our dataset

#Task1

In [ ]:
#Step 1: Remove the records where “user_verified” is “FALSE”. 
verified = amazon_data[amazon_data['user_verified'] == True] #filtering out the 'True' users in the dataset
verified['user_verified'] #displaying the filtered column to check if the filter has been set correctly

In [ ]:
#Step 2: For the remaining records (“user_verified” is “TRUE”), group by created date, and count the number of tweets for each date. 

verified['Month'] = pd.DatetimeIndex(verified['tweet_created_at']).month #using datetime function of pandas to extarct month

#import calender
#verified_true['Month'] = verified_true.apply(lambda x : calender.month_abbr[x])

verified['Month'] = pd.to_datetime(verified['Month'], format='%m').dt.month_name().str.slice(stop=3) #converting it to string format to match the question 
verified['Day'] = pd.DatetimeIndex(verified['tweet_created_at']).day #extracting day using pandas datetime function

In [ ]:
verified['tweet_created_date'] = verified['Month'] + " " + verified['Day'].astype(str) #concatinating the two columns
verified.head() #checking the final result of our date extraction process

In [ ]:
count_tweet = verified.groupby(['tweet_created_date']).size().reset_index(name='Number of Tweets') #Had to name the column to help call it during sorting process
count_tweet #checking to see if the number of tweets have been tabulated correctly

In [ ]:
#Step 3: For the date with highest number of tweets (you can figure it out from step 2), calculate the sum of “favorite_count” and “retweet_count” for each tweet on that day. 
count_tweet_count = count_tweet.sort_values(by='Number of Tweets', ascending=False) #sorting the columns by 'Number of Tweets'
count_tweet_count #We get to know the Jan 3 had the highest number of tweets, i.e. 1536

In [ ]:
highest_tweet = verified[verified.tweet_created_date == 'Jan 3'] #extracting tweets pertinent to Jan 3 from verified_true
highest_tweet #checking to see if all 1536 tweets have been extracted

highest_tweet['tweet_sum'] = highest_tweet['favorite_count'] + highest_tweet['retweet_count'] #calculating the sum of the two columns and tabulating it in a new column 'tweet_sum'
highest_tweet #checking to see if the data has been tabulated correctly

In [ ]:
#Step 3: Then report the text content (“text_”) of the top 100 tweets with highest sum. 
highest_tweet = highest_tweet.sort_values(by = 'tweet_sum', ascending = False)
highest_tweet

In [ ]:
#Step 3: Count the word frequency of the 100 tweets and report the result (Note that data cleaning steps before wordcount can be done outside of Spark operations).
highest_tweet_100 = highest_tweet.nlargest(100, 'tweet_sum') #using the 'nlargest' function in pandas to find the top 100 tweets sorted by 'tweet_sum'
highest_tweet_100 = highest_tweet_100[['tweet_sum', 'text_']] #we add the 'text_' function to add the text to the data frame as well
highest_tweet_100 #checking if the operation is successful

In [ ]:
#spark= SparkSession.builder.getOrCreate()
#spDF = SQLContext.createDataFrame(highest_tweet_100)
#rdd = sc.parallelize([highest_tweet_100])

text = "" #creating a variable to store the list, this would make it easier to pass it through RDD
for i in range (0,100):
    text = text + highest_tweet_100['text_'].values[i] #passing the list to the empty variable

In [ ]:
#Step3: Count the word frequency of the 100 tweets and report the result
sc =SparkContext.getOrCreate()
text_file = sc.parallelize([text]) #passing the text list
word_count = text_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).collect() #using the word-count function of pyspark RDD

In [ ]:
print(word_count) #printing the list generated

In [ ]:
type(word_count) #ensuring type of 'word_count' to convert to spark dataframe

In [ ]:
columns = ["Word", "Frequency"] #defining the column names in our output
word_count_table = spark.createDataFrame(data = word_count, schema = columns) #converting the list to Spark dataframe
word_count_table.show() #displaying the output

#Task2

In [ ]:
#You will use find_text.csv for this task. There are two columns in this document: “id_str” and “text”. The second column is empty. Please find out the text content of each tweet according to “id_str” joining Amazon_Responded_Oct05.csv and fill in the “text” column.
find_text = pd.read_csv('C:/Users/Sachin Chavan/Downloads/find_text.csv') #importing find_text file in pandas dataframe
find_text.head() #checking the dataset

In [ ]:
text_spark = spark.createDataFrame(find_text) #converting find_text to spark dataframe
amazon_string = amazon_data.astype(str) #we need to convert the dataframe to string to avoid errors when converting to spark dataframe
amazon_spark = spark.createDataFrame(amazon_string) #converting amazon_data to spark dataframe

In [ ]:
type(text_spark) #re-verifying if the conversion has been successful

In [ ]:
type(amazon_spark) #re-verifying if the conversion has been successful

In [ ]:
final_output = amazon_spark.join(text_spark,'id_str').select('id_str','text_') #using the join function of pyspark
final_output.show()

In [ ]:
final_output.toPandas().to_csv(('C:/Users/Sachin Chavan/Downloads/HW2_sachin.csv'), index = True, header=True)